<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/New1000popular_repos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install PyGithub

In [7]:
import pandas as pd
from google.colab import drive
from github import Github, RateLimitExceededException
from github.GithubException import UnknownObjectException
from tqdm import tqdm
from datetime import datetime
import time
import os
import json
import pickle
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
new_1000 = pd.read_excel('/content/drive/MyDrive/GHDomains/1000Popular_repos.xlsx', index_col=0)
new = new_1000['Name'].to_list()

In [4]:
old_1000 = pd.read_csv('/content/drive/MyDrive/GHDomains/expanded_frame_final.csv', usecols=['Name'])
old = old_1000['Name'].to_list()

In [5]:
repos_ids = list(set(new) - set(old))

In [6]:
len(repos_ids)

441

In [8]:
with open('/content/drive/MyDrive/GHDomains/new_popular_list.pickle', 'wb') as fp:
    pickle.dump(repos_ids, fp)

In [ ]:
token = input("Enter your Github token:")
g = Github(token, timeout=30, per_page=100)

In [9]:
def catch(c):
    try:
        return c.author.login
    except:
        return c.commit.author.name


def get_commits_data(token, repo_id):

    repo = token.get_repo(repo_id)

    repo_commits_author = []
    repo_commits_message = []
    try:
        
        commits = repo.get_commits()
        repo_commits_author.extend([catch(commit) for commit in commits])
        repo_commits_message.extend([commit.commit.message for commit in commits])
        
        if len(repo_commits_author) != 0 and len(repo_commits_author) !=0:
            return {'Authors:': repo_commits_author, 'Messages:': repo_commits_message}
        else:
            return None
    except:
	    return None

def download_commits(token, repo_id):

    return get_commits_data(token, repo_id)

In [15]:
output_directory_path    = '/content/drive/MyDrive/GHDomains/Downloaded_repos'
repos_already_downloaded = [file_name.replace('.json', '').replace(':','/') for file_name in os.listdir(output_directory_path) if file_name.endswith('.json') and not file_name.startswith('all_repos')]
repos_to_download        = [repo_name for repo_name in repos_ids if repo_name not in repos_already_downloaded]

In [16]:
repos_to_download = ['huginn/huginn',
                      'mastodon/mastodon',
                      'quozd/awesome-dotnet',
                      'kamranahmedse/design-patterns-for-humans',
                      'derailed/k9s',
                      'spmallick/learnopencv',
                      'qishibo/AnotherRedisDesktopManager',
                      'bailicangdu/vue2-elm',
                      'akullpp/awesome-java',
                      'puppeteer/puppeteer',
                      'alibaba/arthas',
                      'atlassian/react-beautiful-dnd',
                      'plotly/dash']

In [17]:
for i, repo_id in enumerate(repos_to_download):
  try:
    print('Downloading repo: ' + repo_id + ' ...')
    print(str(time.asctime(time.localtime(time.time()))) + ' ' + ': #' + str(i+1+len(repos_already_downloaded)) + ' out of ' + str(len(repos_ids)) + ' repos')

    commits = download_commits(g, repo_id)
    repo_id = repo_id.replace('/', ':')
    json.dump(commits, open(output_directory_path + '/' + repo_id + '.json', 'w'))

    print('- '*50)
  except RateLimitExceededException:
    print('sleeping...')
    time.sleep((g.get_rate_limit().core.reset - datetime.today()).seconds + 5)
    print('continue...')
  except UnknownObjectException:
    print(repo_id + ' not found')
    continue

Wed Mar 16 04:22:53 2022 : #441 out of 441 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Wed Mar 16 04:23:03 2022 : #442 out of 441 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Wed Mar 16 04:23:42 2022 : #443 out of 441 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Wed Mar 16 04:23:47 2022 : #444 out of 441 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Wed Mar 16 04:23:48 2022 : #445 out of 441 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Wed Mar 16 04:23:53 2022 : #446 out of 441 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Wed Mar 16 04:24:00 2022 : #447 out of 441 repos
- - - - - - - - - - - - - - - - - - - - - - - - - -